# PyCDT Guessed Charge States

In [4]:
# note this is with doped v1.1.2 which used PyCDT for defect + charge state generation:
from pymatgen.core.structure import Structure
from doped.pycdt.core.defectsmaker import ChargedDefectsStructures

In [60]:
def convert_pycdt_to_doped_name(pycdt_name):
    if pycdt_name.startswith("vac_"):
        return f"v_{pycdt_name.rsplit('_', 1)[-1]}"
    elif pycdt_name.startswith("as_") or pycdt_name.startswith("sub_"):
        return f"{pycdt_name.split('_')[2]}_{pycdt_name.split('_')[4]}"
    elif pycdt_name.startswith("inter_"):
        return f"{pycdt_name.rsplit('_', 1)[-1]}_i"
    else:
        raise ValueError(f"Unknown defect type: {pycdt_name}")

In [48]:
def print_false_pos_neg_hits(false_positives, false_negatives, hits):
    print(f"False positives: {false_positives}")
    print(f"False negatives: {false_negatives}")
    print(f"Hits: {hits}")
    print(f"\nFalse positives: {sum([len(val) for val in false_positives.values()])}")
    print(f"False negatives: {sum([len(val) for val in false_negatives.values()])}")
    print(f"Hits: {sum([len(val) for val in hits.values()])}")

def generate_and_compare_doped_charges(tld_stable_charges, struct=None, cds=None):
    if cds is None:
        cds = ChargedDefectsStructures(struct.get_primitive_structure(), cellmax=struct.num_sites, 
                                       intersites=[struct.get_primitive_structure()[0]])
    defect_dict_list = [defect_dict for key in cds.defects.keys() if key != "bulk" for defect_dict in cds.defects[key]]
    false_positives = {}
    false_negatives = {}
    hits = {}
    for defect, charge_list in tld_stable_charges.items():
        for defect_dict in defect_dict_list:
            if convert_pycdt_to_doped_name(defect_dict["name"]).startswith(defect):
                generated_defect_charges = set(defect_dict["charges"])
                break

        false_negatives[defect] = set(charge_list) - generated_defect_charges
        false_positives[defect] = generated_defect_charges - set(charge_list)
        hits[defect] = (set(charge_list) & generated_defect_charges)
    print_false_pos_neg_hits(false_positives, false_negatives, hits)

### CdTe

In [24]:
CdTe_tld_stable_charges = {
    "v_Cd": list(range(-2, 0 + 1)),
    "v_Te": list(range(0, +2 + 1)),
    "Cd_i": list(range(0, +2 + 1)),
    "Te_i": list(range(0, +2 + 1)),
    "Cd_Te": list(range(0, +2 + 1)),
    "Te_Cd": list(range(-2, +2 + 1)),
}

In [25]:
primitive_cdte = Structure.from_file("../../examples/CdTe/relaxed_primitive_POSCAR")
cdte_cds = ChargedDefectsStructures(primitive_cdte)

Setting up vacancies
Setting up antisites
Searching for Voronoi interstitial sites (this can take a little while)
Found the interstital sites, setting up interstitials
Defects generated:
Vacancies    Potential Charge States    Supercell Site Multiplicity
-----------  -------------------------  -----------------------------
vac_1_Cd     [-2, -1, 0, 1, 2]          1
vac_2_Te     [-2, -1, 0, 1, 2]          1 

Substitutions    Potential Charge States    Supercell Site Multiplicity
---------------  -------------------------  -----------------------------
as_1_Cd_on_Te    [-2, -1, 0, 1, 2, 3, 4]    1
as_1_Te_on_Cd    [-2, -1, 0, 1, 2, 3, 4]    1 

Interstitials    Potential Charge States        Supercell Site Multiplicity
---------------  -----------------------------  -----------------------------
inter_1_Cd       [0, 1, 2]                      4
inter_2_Cd       [0, 1, 2]                      1
inter_3_Cd       [0, 1, 2]                      1
inter_4_Te       [-2, -1, 0, 1, 2, 3, 4, 5, 6

In [30]:
generate_and_compare_doped_charges(CdTe_tld_stable_charges, cds=cdte_cds)

False positives: {'v_Cd': {1, 2}, 'v_Te': {-1, -2}, 'Cd_i': set(), 'Te_i': {3, 4, 5, 6, -2, -1}, 'Cd_Te': {3, 4, -1, -2}, 'Te_Cd': {3, 4}}
False negatives: {'v_Cd': set(), 'v_Te': set(), 'Cd_i': set(), 'Te_i': set(), 'Cd_Te': set(), 'Te_Cd': set()}
Hits: {'v_Cd': {0, -2, -1}, 'v_Te': {0, 1, 2}, 'Cd_i': {0, 1, 2}, 'Te_i': {0, 1, 2}, 'Cd_Te': {0, 1, 2}, 'Te_Cd': {0, 1, 2, -2, -1}}

False positives: 16
False negatives: 0
Hits: 20


### ZnS

In [31]:
zns_tld_stable_charges = {
    "v_Zn": list(range(-2, +2 + 1)), 
    "v_S": list(range(-2, +2 + 1)), 
    "Zn_i": list(range(0, +2 + 1)),
    "S_i": list(range(0, +2 + 1)), 
    "Zn_S": list(range(-2, +2 + 1)), 
    "S_Zn": list(range(-2, +2 + 1)),
}

In [33]:
supercell_zns = Structure.from_file("../../tests/data/non_diagonal_ZnS_supercell_POSCAR")
cds = ChargedDefectsStructures(supercell_zns)

Setting up vacancies
Setting up antisites
Searching for Voronoi interstitial sites (this can take a little while)
Found the interstital sites, setting up interstitials
Defects generated:
Vacancies    Potential Charge States    Supercell Site Multiplicity
-----------  -------------------------  -----------------------------
vac_1_Zn     [-2, -1, 0, 1, 2]          32
vac_2_S      [-2, -1, 0, 1, 2]          32 

Substitutions    Potential Charge States    Supercell Site Multiplicity
---------------  -------------------------  -----------------------------
as_1_S_on_Zn     [-2, -1, 0, 1, 2, 3, 4]    32
as_1_Zn_on_S     [-2, -1, 0, 1, 2, 3, 4]    32 

Interstitials    Potential Charge States        Supercell Site Multiplicity
---------------  -----------------------------  -----------------------------
inter_1_Zn       [0, 1, 2]                      1
inter_2_Zn       [0, 1, 2]                      1
inter_3_Zn       [0, 1, 2]                      1
inter_4_Zn       [0, 1, 2]               

In [35]:
generate_and_compare_doped_charges(zns_tld_stable_charges, cds=cds)

False positives: {'v_Zn': set(), 'v_S': set(), 'Zn_i': set(), 'S_i': {3, 4, 5, 6, -2, -1}, 'Zn_S': {3, 4}, 'S_Zn': {3, 4}}
False negatives: {'v_Zn': set(), 'v_S': set(), 'Zn_i': set(), 'S_i': set(), 'Zn_S': set(), 'S_Zn': set()}
Hits: {'v_Zn': {0, 1, 2, -2, -1}, 'v_S': {0, 1, 2, -2, -1}, 'Zn_i': {0, 1, 2}, 'S_i': {0, 1, 2}, 'Zn_S': {0, 1, 2, -2, -1}, 'S_Zn': {0, 1, 2, -2, -1}}

False positives: 10
False negatives: 0
Hits: 26


### YTOS

In [42]:
ytos_tld_stable_charges = {
    "v_O": list(range(0, +2 + 1)),
    "v_S": list(range(0, +2 + 1)),
    "v_Y": list(range(-3, 0 + 1)),
    "v_Ti": list(range(-4, 0 + 1)),
    "O_i": list(range(-2, 0 + 1)),
    "Ti_i": list(range(0, +4 + 1)),
    "S_i": list(range(0, +4 + 1)),  
    "Y_i": list(range(0, +3 + 1)),
    "O_S": list(range(0, 0 + 1)),
    "S_O": list(range(0, 0 + 1)),
    "S_Ti": list(range(0, +2 + 1)),  # not stable in a negative charge state?
    "S_Y": list(range(0, +3 + 1)),  # not stable in a negative charge state?
    "Ti_Y": list(range(0, +1 + 1)),
    "Y_Ti": list(range(-1, 0 + 1)),
}

In [45]:
ytos_struc = Structure.from_file("../../examples/YTOS/Bulk/POSCAR")  # Y2Ti2S2O5
generate_and_compare_doped_charges(ytos_tld_stable_charges, ytos_struc.get_primitive_structure())

Setting up vacancies
Setting up antisites
Setting up interstitials from intersites
Defects generated:
Vacancies    Potential Charge States          Supercell Site Multiplicity
-----------  -------------------------------  -----------------------------
vac_1_O      [-2, -1, 0, 1, 2]                1
vac_2_O      [-2, -1, 0, 1, 2]                4
vac_3_S      [-2, -1, 0, 1, 2]                2
vac_4_Ti     [-4, -3, -2, -1, 0, 1, 2, 3, 4]  2
vac_5_Y      [-3, -2, -1, 0, 1, 2, 3]         2 

Substitutions    Potential Charge States    Supercell Site Multiplicity
---------------  -------------------------  -----------------------------
as_1_S_on_O      [-2, -1, 0, 1, 2, 3, 4]    1
as_2_S_on_O      [-2, -1, 0, 1, 2, 3, 4]    4
as_3_S_on_Ti     [-2, -1, 0, 1, 2, 3, 4]    2
as_4_S_on_Y      [-2, -1, 0, 1, 2, 3, 4]    2
as_1_O_on_S      [-2, -1, 0, 1, 2, 3, 4]    2
as_2_O_on_Ti     [-2, -1, 0, 1, 2, 3, 4]    2
as_3_O_on_Y      [-2, -1, 0, 1, 2, 3, 4]    2
as_1_Ti_on_O     [-2, -1, 0, 1, 2, 3, 

### LMNO

In [49]:
lmno_tld_stable_charges = {
    "v_Li": list(range(-1, 0 + 1)),  # only went up to -1, but extend to 0 (always included)
    "v_Mn": list(range(-3, 0 + 1)),  # only went up to -1, but extend to 0 (always included)
    "v_Ni": list(range(-2, 0 + 1)),  # only went up to -1, but extend to 0 (always included)
    "v_O": list(range(0, 1 + 1)),
    "Li_i": list(range(0, 1 + 1)),
    "Mn_i": list(range(0, +3 + 1)),
    "Ni_i": list(range(0, +2 + 1)),
    "O_i": list(range(-1, 0 + 1)),
    "Li_Mn": list(range(-2, 0 + 1)),  # only went up to -1, but extend to 0 (always included)
    "Mn_Li": list(range(1, 1 + 1)),
    "Ni_Li": list(range(0, +1 + 1)),
    "Li_Ni": list(range(-1, 0 + 1)),  # only went up to -1, but extend to 0 (always included)
    "Mn_Ni": list(range(0, +1 + 1)),
    "Ni_Mn": list(range(-1, 0 + 1)),
}

In [50]:
lmno_struct = Structure.from_file("../../tests/data/Li2Mn3NiO8_POSCAR")
generate_and_compare_doped_charges(lmno_tld_stable_charges, lmno_struct)

Setting up vacancies
Setting up antisites
Setting up interstitials from intersites
Defects generated:
Vacancies    Potential Charge States          Supercell Site Multiplicity
-----------  -------------------------------  -----------------------------
vac_1_Li     [-1, 0, 1]                       8
vac_2_Mn     [-4, -3, -2, -1, 0, 1, 2, 3, 4]  12
vac_3_Ni     [-2, -1, 0, 1, 2]                4
vac_4_O      [-2, -1, 0, 1, 2]                24
vac_5_O      [-2, -1, 0, 1, 2]                8 

Substitutions    Potential Charge States     Supercell Site Multiplicity
---------------  --------------------------  -----------------------------
as_1_O_on_Li     [-2, -1, 0, 1, 2, 3, 4, 5]  8
as_2_O_on_Mn     [-2, -1, 0, 1, 2, 3, 4, 5]  12
as_3_O_on_Ni     [-2, -1, 0, 1, 2, 3, 4, 5]  4
as_1_Mn_on_Li    [-2, -1, 0, 1, 2, 3, 4, 5]  8
as_2_Mn_on_Ni    [-2, -1, 0, 1, 2, 3, 4, 5]  4
as_3_Mn_on_O     [-2, -1, 0, 1, 2, 3, 4, 5]  24
as_4_Mn_on_O     [-2, -1, 0, 1, 2, 3, 4, 5]  8
as_1_Li_on_Mn    [-2, -1,

### Na$_2$FePO$_4$F

In [51]:
na2fepo4f_tld_stable_charges = {
    "v_Na": list(range(-1, 0 + 1)),
    "v_Fe": list(range(-2, 0 + 1)),
    "v_P": list(range(-5, 0 + 1)),
    "v_O": list(range(0, +2 + 1)),
    "v_F": list(range(0, +1 + 1)),
    "Na_i": list(range(0, +1 + 1)),
    "Fe_i": list(range(0, +2 + 1)),
    "P_i": list(range(0, +5 + 1)),
    "O_i": list(range(-2, 0 + 1)),
    "F_i": list(range(-1, 0 + 1)),
    "Na_Fe": list(range(-1, 0 + 1)),
    "Na_P": list(range(-4, 0 + 1)),
    "Na_O": list(range(0, +1 + 1)), # +2, +3 -> tested but not stable
    "Na_F": list(range(0, +2 + 1)),
    "Fe_Na": list(range(0, +1 + 1)),
    "Fe_P": list(range(-2, 0 + 1)), # -3
    "Fe_O": list(range(0, +2 + 1)), # +3, +4
    "Fe_F": list(range(0, +2 + 1)), # +3
    "P_Na": list(range(0, +1 + 1)), # +2, +3. +4
    "P_Fe": list(range(0, +3 + 1)), # +1, +2
    "P_O": list(range(0, +1 + 1)), # +2, +3, +4, +5, +6, +7
    "P_F": list(range(0, +1 + 1)), # +2, +3, +4, +5, +6
    "O_Na": list(range(-2, 0 + 1)), # -3
    "O_Fe": list(range(-2, 0 + 1)), # -3, -4
    "O_P": list(range(-2, 0 + 1)), # -3, -4, -5, -6, -7
    "O_F": list(range(-1, 0 + 1)), # 0
    "F_Na": list(range(-1, 0 + 1)), # -2
    "F_Fe": list(range(-2, 0 + 1)), # -3
    "F_P": list(range(-2, 0 + 1)), # -3, -4, -5, -6
    "F_O": list(range(0, +1 + 1)),
}

In [52]:
mp_na2fepo4f = Structure.from_file("data/Na2FePO4F_POSCAR")
generate_and_compare_doped_charges(na2fepo4f_tld_stable_charges, mp_na2fepo4f)

Setting up vacancies
Setting up antisites
Setting up interstitials from intersites
Defects generated:
Vacancies    Potential Charge States                 Supercell Site Multiplicity
-----------  --------------------------------------  -----------------------------
vac_1_Na     [-1, 0, 1]                              8
vac_2_Na     [-1, 0, 1]                              8
vac_3_Fe     [-2, -1, 0, 1, 2]                       8
vac_4_P      [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]  8
vac_5_O      [-2, -1, 0, 1, 2]                       8
vac_6_O      [-2, -1, 0, 1, 2]                       8
vac_7_O      [-2, -1, 0, 1, 2]                       8
vac_8_O      [-2, -1, 0, 1, 2]                       8
vac_9_F      [-1, 0, 1]                              4
vac_10_F     [-1, 0, 1]                              4 

Substitutions    Potential Charge States    Supercell Site Multiplicity
---------------  -------------------------  -----------------------------
as_1_O_on_Na     [-3, -2, -1, 0, 1, 

### BaBi2O$_6$

In [53]:
babio_tld_stable_charges = {
    "v_Ba": list(range(-2, 0 + 1)),
    "v_Bi": list(range(-2, 0 + 1)),
    "v_O": list(range(0, +2 + 1)),

    "Ba_i": list(range(0, +2 + 1)),
    "Bi_i": list(range(+1, +2 + 1)),
    "O_i": list(range(0, 0 + 1)),

    "Ba_Bi": list(range(-2, 0 + 1)),
    "Bi_Ba": list(range(+1, +1 + 1)),

    "La_i": list(range(0, +3 + 1)),
    "La_Ba": list(range(+1, +1 + 1)),
    "La_Bi": list(range(-2, 0 + 1)),

    "F_i": list(range(-1, +1 + 1)),
    "F_O": list(range(+1, +1 + 1)),
}

In [58]:
oqmd_babio = Structure.from_file("data/BaBi2O6_POSCAR")
cds = ChargedDefectsStructures(oqmd_babio, cellmax=oqmd_babio.num_sites, 
                               substitutions={"Ba": ["La"], "O": ["F"], "Bi": ["La"]}, 
                               interstitial_elements=["F", "La", "Ba", "Bi", "O"])

Setting up vacancies
Setting up antisites
Searching for Voronoi interstitial sites (this can take a little while)
Found the interstital sites, setting up interstitials
Defects generated:
Vacancies    Potential Charge States                 Supercell Site Multiplicity
-----------  --------------------------------------  -----------------------------
vac_1_Ba     [-2, -1, 0, 1, 2]                       1
vac_2_Bi     [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]  2
vac_3_O      [-2, -1, 0, 1, 2]                       6 

Substitutions    Potential Charge States    Supercell Site Multiplicity
---------------  -------------------------  -----------------------------
sub_1_La_on_Ba   [-2, -1, 0, 1, 2]          1
sub_3_F_on_O     [-2, -1, 0, 1, 2]          6
sub_2_La_on_Bi   [-2, -1, 0, 1]             2
as_1_Ba_on_Bi    [-2, -1, 0, 1]             2
as_2_Ba_on_O     [-2, -1, 0, 1]             6
as_1_O_on_Ba     [-2, -1, 0, 1]             1
as_2_O_on_Bi     [-2, -1, 0, 1]             2
as_1_Bi_on_Ba 

In [61]:
generate_and_compare_doped_charges(babio_tld_stable_charges, cds=cds)

False positives: {'v_Ba': {1, 2}, 'v_Bi': {1, 2, 3, 4, 5, -5, -4, -3}, 'v_O': {-1, -2}, 'Ba_i': set(), 'Bi_i': {0, 3}, 'O_i': {-2, -1}, 'Ba_Bi': {1}, 'Bi_Ba': {0, -2, -1}, 'La_i': set(), 'La_Ba': {0, 2, -1, -2}, 'La_Bi': {1}, 'F_i': set(), 'F_O': {0, 2, -1, -2}}
False negatives: {'v_Ba': set(), 'v_Bi': set(), 'v_O': set(), 'Ba_i': set(), 'Bi_i': set(), 'O_i': set(), 'Ba_Bi': set(), 'Bi_Ba': set(), 'La_i': set(), 'La_Ba': set(), 'La_Bi': set(), 'F_i': {1}, 'F_O': set()}
Hits: {'v_Ba': {0, -2, -1}, 'v_Bi': {0, -2, -1}, 'v_O': {0, 1, 2}, 'Ba_i': {0, 1, 2}, 'Bi_i': {1, 2}, 'O_i': {0}, 'Ba_Bi': {0, -2, -1}, 'Bi_Ba': {1}, 'La_i': {0, 1, 2, 3}, 'La_Ba': {1}, 'La_Bi': {0, -2, -1}, 'F_i': {0, -1}, 'F_O': {1}}

False positives: 29
False negatives: 1
Hits: 30


### Sb$_2$O$_5$

In [62]:
sb2o5_tld_stable_charges = {
"v_Sb": list(range(-4, 0 + 1)),
"v_O": list(range(0, +2 + 1)),
"Sb_i": list(range(+2, +5 + 1)),
"O_i": list(range(0, 0 + 1)),
"Sb_O": list(range(+1, +1 + 1)),
"F_O": list(range(+1, +1 + 1)),
"F_i": list(range(-1, +1 + 1)),
}

In [63]:
sb2o5 = Structure.from_file("data/Sb2O5_POSCAR")
cds = ChargedDefectsStructures(sb2o5, cellmax=sb2o5.num_sites,
                               substitutions={"O": ["F"]},
                              interstitial_elements=["Sb", "O", "F"])
generate_and_compare_doped_charges(sb2o5_tld_stable_charges, cds=cds)

Setting up vacancies
Setting up antisites
Searching for Voronoi interstitial sites (this can take a little while)
Found the interstital sites, setting up interstitials
Defects generated:
Vacancies    Potential Charge States                 Supercell Site Multiplicity
-----------  --------------------------------------  -----------------------------
vac_1_Sb     [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]  8
vac_2_O      [-2, -1, 0, 1, 2]                       4
vac_3_O      [-2, -1, 0, 1, 2]                       8
vac_4_O      [-2, -1, 0, 1, 2]                       8 

Substitutions    Potential Charge States    Supercell Site Multiplicity
---------------  -------------------------  -----------------------------
sub_2_F_on_O     [-2, -1, 0, 1, 2]          4
sub_3_F_on_O     [-2, -1, 0, 1, 2]          8
sub_4_F_on_O     [-2, -1, 0, 1, 2]          8
as_1_O_on_Sb     [-3, -2, -1, 0, 1, 2, 3]   8
as_1_Sb_on_O     [-3, -2, -1, 0, 1, 2, 3]   4
as_2_Sb_on_O     [-3, -2, -1, 0, 1, 2, 3]   8
as_3_

### Al$_2$O$_3$

In [64]:
al2o3_tld_stable_charges = {
"v_Al": list(range(-3, 0, + 1)),
"v_O": list(range(0, +2 + 1)),
"O_i": list(range(-2, 0 + 1)),
"Al_i": list(range(0, +3 + 1)),
}

In [65]:
al2o3 = Structure.from_file("data/Al2O3_POSCAR")
generate_and_compare_doped_charges(al2o3_tld_stable_charges, al2o3)

Setting up vacancies
Setting up antisites
Setting up interstitials from intersites
Defects generated:
Vacancies    Potential Charge States    Supercell Site Multiplicity
-----------  -------------------------  -----------------------------
vac_1_Al     [-3, -2, -1, 0, 1, 2, 3]   4
vac_2_O      [-2, -1, 0, 1, 2]          6 

Substitutions    Potential Charge States    Supercell Site Multiplicity
---------------  -------------------------  -----------------------------
as_1_O_on_Al     [-2, -1, 0, 1]             4
as_1_Al_on_O     [-2, -1, 0, 1]             6 

Interstitials    Potential Charge States    Supercell Site Multiplicity
---------------  -------------------------  -----------------------------
inter_1_Al       [0, 1, 2, 3]               4
inter_1_O        [-2, -1, 0]                4 

False positives: {'v_Al': {0, 1, 2, 3}, 'v_O': {-1, -2}, 'O_i': set(), 'Al_i': set()}
False negatives: {'v_Al': set(), 'v_O': set(), 'O_i': set(), 'Al_i': set()}
Hits: {'v_Al': {-3, -2, -1}, 'v_

### AlSbO$_4$

In [66]:
alsbo4_tld_stable_charges = {
    "v_O": list(range(0, 2 + 1)),
    "v_Sb": list(range(-5, 0 + 1)),
    "v_Al": list(range(-3, 0 + 1)),
    "Sb_Al": list(range(-2, 0 +1)),
    "Al_Sb": list(range(-2, 0 + 1)),
}

In [67]:
alsbo4 = Structure.from_file("data/AlSbO4_POSCAR")
generate_and_compare_doped_charges(alsbo4_tld_stable_charges, alsbo4)

Setting up vacancies
Setting up antisites
Setting up interstitials from intersites
Defects generated:
Vacancies    Potential Charge States                 Supercell Site Multiplicity
-----------  --------------------------------------  -----------------------------
vac_1_Al     [-3, -2, -1, 0, 1, 2, 3]                1
vac_2_O      [-2, -1, 0, 1, 2]                       2
vac_3_O      [-2, -1, 0, 1, 2]                       2
vac_4_Sb     [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]  1 

Substitutions    Potential Charge States    Supercell Site Multiplicity
---------------  -------------------------  -----------------------------
as_1_O_on_Al     [-3, -2, -1, 0, 1, 2, 3]   1
as_2_O_on_Sb     [-3, -2, -1, 0, 1, 2, 3]   1
as_1_Sb_on_Al    [-3, -2, -1, 0, 1, 2, 3]   1
as_2_Sb_on_O     [-3, -2, -1, 0, 1, 2, 3]   2
as_3_Sb_on_O     [-3, -2, -1, 0, 1, 2, 3]   2
as_1_Al_on_O     [-3, -2, -1, 0, 1, 2, 3]   2
as_2_Al_on_O     [-3, -2, -1, 0, 1, 2, 3]   2
as_3_Al_on_Sb    [-3, -2, -1, 0, 1, 2, 3]   

### Sb$_2$Se$_3$

In [68]:
sb2se3_tld_stable_charges = {
    "v_Se": list(range(-2, +2 + 1)),
    "v_Sb": list(range(-3, +1 + 1)),
    "Sb_Se": list(range(-1, +3 +1)),
    "Se_Sb": list(range(-1, +1 +1)),
    "Sb_i": list(range(-1, +3 +1)),
    "Se_i": list(range(0, +4 + 1)),
}

In [70]:
sb2se3 = Structure.from_file("../../tests/data/Sb2Se3_bulk_supercell_POSCAR")
generate_and_compare_doped_charges(sb2se3_tld_stable_charges, sb2se3)

Setting up vacancies
Setting up antisites
Setting up interstitials from intersites
Defects generated:
Vacancies    Potential Charge States    Supercell Site Multiplicity
-----------  -------------------------  -----------------------------
vac_1_Sb     [-3, -2, -1, 0, 1, 2, 3]   4
vac_2_Sb     [-3, -2, -1, 0, 1, 2, 3]   4
vac_3_Se     [-2, -1, 0, 1, 2]          4
vac_4_Se     [-2, -1, 0, 1, 2]          4
vac_5_Se     [-2, -1, 0, 1, 2]          4 

Substitutions    Potential Charge States      Supercell Site Multiplicity
---------------  ---------------------------  -----------------------------
as_1_Se_on_Sb    [-3, -2, -1, 0, 1, 2, 3, 4]  4
as_2_Se_on_Sb    [-3, -2, -1, 0, 1, 2, 3, 4]  4
as_1_Sb_on_Se    [-3, -2, -1, 0, 1, 2, 3, 4]  4
as_2_Sb_on_Se    [-3, -2, -1, 0, 1, 2, 3, 4]  4
as_3_Sb_on_Se    [-3, -2, -1, 0, 1, 2, 3, 4]  4 

Interstitials    Potential Charge States         Supercell Site Multiplicity
---------------  ------------------------------  -----------------------------


### Se

In [71]:
se_tld_stable_charges = {
    "v_Se": list(range(-2, +1 + 1)),  # +1 just about
    "Se_i": list(range(-2, 0 + 1)),  # -2 just about
    "F_Se": list(range(-1, +1 + 1)),
    "F_i": list(range(-1, +1 + 1)),
    "Br_Se": list(range(-1, +1 + 1)),
    "Br_i": list(range(-1, +1 + 1)),
    "Cl_Se": list(range(-1, +1 + 1)),
    "Cl_i": list(range(-1, +1 + 1)),
    "H_Se": list(range(-1, +1 + 1)),
    "H_i": list(range(-1, +1 + 1)),
    "O_Se": list(range(-2, 0 + 1)),  # -2 just about
    "O_i": list(range(-2, 0 + 1)),
    "S_Se": list(range(-1, 0 + 1)),
    "S_i": list(range(0, 0 + 1)),  # -2 at CBM
    "Te_Se": list(range(-1, 0 + 1)),  # -2 for a tiny bit below CBM
    "Te_i": list(range(-2, 0 + 1)),  # -2 for a small bit below CBM
}

In [72]:
se = Structure.from_file("data/Se_HSE06_D3_CONTCAR")
cds = ChargedDefectsStructures(se, interstitial_elements=["F", "Br", "Cl", "H", "O", "S", "Te"],
                              substitutions={"Se":["F", "Br", "Cl", "H", "O", "S", "Te"]})

Setting up vacancies
Setting up antisites
Searching for Voronoi interstitial sites (this can take a little while)
Found the interstital sites, setting up interstitials
Defects generated:
Vacancies    Potential Charge States                        Supercell Site Multiplicity
-----------  ---------------------------------------------  -----------------------------
vac_1_Se     [-6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6]  3 

Substitutions    Potential Charge States    Supercell Site Multiplicity
---------------  -------------------------  -----------------------------
sub_1_F_on_Se    [-2, -1, 0, 1, 2]          3
sub_1_Br_on_Se   [-1, 0, 1, 2, 3, 4]        3
sub_1_Cl_on_Se   [-1, 0, 1, 2, 3, 4]        3
sub_1_H_on_Se    [-2, -1, 0, 1, 2]          3
sub_1_O_on_Se    [-2, -1, 0, 1]             3
sub_1_S_on_Se    [-2, -1, 0, 1, 2, 3]       3
sub_1_Te_on_Se   [-2, -1, 0, 1, 2, 3]       3 

Interstitials    Potential Charge States        Supercell Site Multiplicity
---------------  --------

In [73]:
generate_and_compare_doped_charges(se_tld_stable_charges, cds=cds)

False positives: {'v_Se': {2, 3, 4, 5, 6, -6, -5, -4, -3}, 'Se_i': {1, 2, 3, 4, 5, 6, -6, -5, -4, -3}, 'F_Se': {2, -2}, 'F_i': set(), 'Br_Se': {2, 3, 4}, 'Br_i': {2, 3, 4, 5, 6, 7}, 'Cl_Se': {2, 3, 4}, 'Cl_i': {2, 3, 4, 5, 6, 7}, 'H_Se': {2, -2}, 'H_i': set(), 'O_Se': {1}, 'O_i': set(), 'S_Se': {1, 2, 3, -2}, 'S_i': {1, 2, 3, 4, 5, 6, -1, -2}, 'Te_Se': {1, 2, 3, -2}, 'Te_i': {1, 2, 3, 4, 5, 6}}
False negatives: {'v_Se': set(), 'Se_i': set(), 'F_Se': set(), 'F_i': {1}, 'Br_Se': set(), 'Br_i': set(), 'Cl_Se': set(), 'Cl_i': set(), 'H_Se': set(), 'H_i': set(), 'O_Se': set(), 'O_i': set(), 'S_Se': set(), 'S_i': set(), 'Te_Se': set(), 'Te_i': set()}
Hits: {'v_Se': {0, 1, -2, -1}, 'Se_i': {0, -2, -1}, 'F_Se': {0, 1, -1}, 'F_i': {0, -1}, 'Br_Se': {0, 1, -1}, 'Br_i': {0, 1, -1}, 'Cl_Se': {0, 1, -1}, 'Cl_i': {0, 1, -1}, 'H_Se': {0, 1, -1}, 'H_i': {0, 1, -1}, 'O_Se': {0, -2, -1}, 'O_i': {0, -2, -1}, 'S_Se': {0, -1}, 'S_i': {0}, 'Te_Se': {0, -1}, 'Te_i': {0, -2, -1}}

False positives: 64
False ne

### Zn$_3$P$_2$:

In [74]:
zn3p2_tld_stable_charges = {  # from Yihuang and Geoffroy's: https://arxiv.org/abs/2306.13583
    "v_Zn": list(range(-2, 0 + 1)),
    "v_P": list(range(-1, +1 + 1)),
    "Zn_i": list(range(0, +2 + 1)),
    "P_i": list(range(-1, +3 + 1)),
    "Zn_P": list(range(-2, +3 + 1)),  # -2 just below CBM...
    "P_Zn": list(range(-1, +3 + 1)),
}

In [75]:
zn3p2 = Structure.from_file("data/Zn3P2_POSCAR")
generate_and_compare_doped_charges(zn3p2_tld_stable_charges, zn3p2)

Setting up vacancies
Setting up antisites
Setting up interstitials from intersites
Defects generated:
Vacancies    Potential Charge States    Supercell Site Multiplicity
-----------  -------------------------  -----------------------------
vac_1_Zn     [-2, -1, 0, 1, 2]          8
vac_2_Zn     [-2, -1, 0, 1, 2]          8
vac_3_Zn     [-2, -1, 0, 1, 2]          8
vac_4_P      [-3, -2, -1, 0, 1, 2, 3]   4
vac_5_P      [-3, -2, -1, 0, 1, 2, 3]   4
vac_6_P      [-3, -2, -1, 0, 1, 2, 3]   8 

Substitutions    Potential Charge States    Supercell Site Multiplicity
---------------  -------------------------  -----------------------------
as_1_Zn_on_P     [-3, -2, -1, 0, 1, 2, 3]   4
as_2_Zn_on_P     [-3, -2, -1, 0, 1, 2, 3]   4
as_3_Zn_on_P     [-3, -2, -1, 0, 1, 2, 3]   8
as_1_P_on_Zn     [-3, -2, -1, 0, 1, 2, 3]   8
as_2_P_on_Zn     [-3, -2, -1, 0, 1, 2, 3]   8
as_3_P_on_Zn     [-3, -2, -1, 0, 1, 2, 3]   8 

Interstitials    Potential Charge States         Supercell Site Multiplicity
-----

## Overall

In [85]:
pycdt_results = {"CdTe": (16, 0, 20),  # remember this is biased to have low false negatives!
                 "Y$_2$Ti$_2$S$_2$O$_5$": (44, 0, 45),
                 "ZnS": (10, 0, 26),
                 "Li$_2$Mn$_3$NiO$_8$": (52, 0, 34),
                 "BaBi$_2$O$_6$": (29, 1, 30), 
                 "Sb$_2$O$_5$": (25, 1, 17),
                 "Al$_2$O$_3$": (6, 0, 13),
                 "AlSbO$_4$": (18, 0, 19),
                 "Sb$_2$Se$_3$": (18, 0, 28),
                 "Zn$_3$P$_2$": (13, 0, 25),
                 "Na$_2$FePO$_4$F": (104, 1, 85),
                 "Se": (64, 1, 44),  
                 }